# Project 3

In this project, you will perform a logistic regression on admissions data

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np

C:\Users\ystrano\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna()
df.head()

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397 entries, 0 to 399
Data columns (total 4 columns):
admit       397 non-null int64
gre         397 non-null float64
gpa         397 non-null float64
prestige    397 non-null float64
dtypes: float64(3), int64(1)
memory usage: 15.5 KB


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables.  Look at the documentation for pd.crosstab

In [4]:
prestige = pd.crosstab(index=df['prestige'], columns='count')
prestige

col_0,count
prestige,
1.0,61
2.0,148
3.0,121
4.0,67


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [5]:
prestige_dummies = pd.get_dummies(df['prestige'])
prestige_dummies.head()

,1.0,2.0,3.0,4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: 3 (always 1 less than the number of calss variables) 

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [19]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(prestige_dummies)
handCalc.head()

,admit,gre,gpa,1.0,2.0,3.0,4.0
0,0,380.0,3.61,0,0,1,0
1,1,660.0,3.67,0,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,0,0,0,1
4,0,520.0,2.93,0,0,0,1


In [50]:
len(handCalc['admit'])

397

In [47]:
len(handCalc[handCalc['admit']==0])

271

In [42]:
len(handCalc[handCalc['admit']==1])

126

In [41]:
handCalc['admit'].sum()

126

In [43]:
len(handCalc[handCalc[1.0]==0])

336

In [44]:
len(handCalc[handCalc[1.0]==1])

61

In [28]:
handCalc[1.0].sum()

61

In [35]:
prestige = pd.crosstab(index=handCalc[1.0], columns="count")
prestige

col_0,count
1.0,
0,336
1,61


In [59]:
comb = pd.crosstab(index=handCalc['admit'], columns=[handCalc[1.0],handCalc[2.0],handCalc[3.0],handCalc[4.0]], margins=True)
comb.columns = ['prestige 1','prestige 2','prestige 3','prestige 4', 'rowtotal']
comb.index= ['admit 0','admit 1', 'coltotal']
comb

,prestige 1,prestige 2,prestige 3,prestige 4,rowtotal
admit 0,55,93,95,28,271
admit 1,12,28,53,33,126
coltotal,0,0,0,0,397


In [57]:
# crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
comb = pd.crosstab(index=handCalc['admit'], columns=handCalc[1.0])
comb.columns = [1, 0]
comb.index= ['admit 0','admit 1']
comb

,1,0
admit 0,243,28
admit 1,93,33


In [58]:
# crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
comb = pd.crosstab(index=handCalc[1.0], columns=handCalc['admit'])
comb.columns = ['admit 0','admit 1']
comb.index= [1, 0]
comb

,admit 0,admit 1
1,243,93
0,28,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

#### 3.3 Calculate the odds ratio

#### 3.4 Write this finding in a sentenance: 

Answer: 

#### 3.5 Print the cross tab for prestige_4

#### 3.6 Calculate the Odds Ratio 

#### 3.7 Write this finding in a sentence

Answer:

## Part 4. Analysis

In [12]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']

## Dropping one of the dummy columns
data = df[cols_to_keep].join(prestige_dummies.iloc[:, 1:])
data.head()

,admit,gre,gpa,2.0,3.0,4.0
0,0,380.0,3.61,0,1,0
1,1,660.0,3.67,0,1,0
2,1,800.0,4.00,0,0,0
3,1,640.0,3.19,0,0,1
4,0,520.0,2.93,0,0,1


#### 4.1 Create the X and Y variables

#### 4.2 Fit the model - 

 - Load sklearn's logistic regression
 - Create the regression object
 - Fit the model

#### 4.3 Print the coefficients

#### 4.4 Calculate the odds ratios of the coeffiencents

hint 1: np.exp(X)

#### 4.5 Interpret the OR of Prestige_2

Answer: 

#### 4.6 Interpret the OR of GPA

Answer: 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.